### --- Day 14: Regolith Reservoir ---

Puzzle description redacted as-per Advent of Code guidelines

You may find the puzzle description at: https://adventofcode.com/2022/day/14

In [2]:
#!import ../Utils.ipynb

In [3]:
var inputLines = LoadPuzzleInput(2022, 14);
WriteLines(inputLines);

Loading puzzle file: Day14.txt
Total lines: 176
Max line length: 266

500,30 -> 504,30
503,28 -> 507,28
477,136 -> 477,133 -> 477,136 -> 479,136 -> 479,135 -> 479,136 -> 481,136 -> 481,127 -> 481,136 -> 483,136 -> 483,131 -> 483,136 -> 485,136 -> 485,129 -> 485,136 -> 487,136 -> 487,135 -> 487,136
496,97 -> 496,99 -> 488,99 -> 488,103 -> 508,103 -> 508,99 -> 502,99 -> 502,97
537,66 -> 541,66


In [4]:
string[] testInputLines = 
[
    "498,4 -> 498,6 -> 496,6",
    "503,4 -> 502,4 -> 502,9 -> 494,9",
];

So, I think for this one we're going to need our grid.

Then, we'll need a function to draw lines in grid space.

Finally a function to simulate dropping items until they can't move any more (or fall off the edge)

In [5]:
// Calculate all points in a straight line
IEnumerable<Point> DrawLine(Point from, Point to)
{
    var diff = to - from;
    Point direction = diff switch {
        (0, var y) => (0, y / Math.Abs(y)),
        (var x, _) => (x / Math.Abs(x), 0)
    };

    var steps = Math.Abs(diff.X + diff.Y) + 1;

    return Enumerable.Range(0, steps).Select(i => from + direction * i);
}

In [6]:
using SandGrid = SCG.Dictionary<Point, char>;
const char Rock = '#';
const char Sand = 'o';

In [7]:
Point[] directions = [Down, Down + Left, Down + Right];

SandGrid ParseSandGrid(string[] inputLines)
{
    SandGrid grid = new();

    foreach (var line in inputLines)
    {
        var linePoints = line.Split(" -> ").Select(ParsePoint).ToList();
        foreach (var (a, b) in linePoints.Zip(linePoints.Skip(1)))
        foreach (var x in DrawLine(a, b))
        {
            grid[x] = Rock;
        }
    }

    return grid;

    Point ParsePoint(string f) => (f.ParseInts().First(), f.ParseInts().Last());
}

In [8]:
void DropSand(SandGrid grid)
{
    Point sand = (500, 0);
    var floor = grid.Keys.Max(p => p.Y);

    while (!grid.ContainsKey(sand))
    {
        // Fallen off?
        if (sand.Y > floor) { return; }

        // See if we can drop down
        var next = directions.Select(dir => sand + dir)
            .FirstOrDefault(n => !grid.ContainsKey(n));
        
        // Nowhere left to fall?
        if (next == default) { break; }

        // Still falling
        sand = next;
    }

    grid[sand] = Sand;
}

In [9]:
int DropAndCountSand(string[] inputLines, Action<SandGrid> dropSand)
{
    var grid = ParseSandGrid(inputLines);

    var current = -2;
    var next = -1;
    while (next != current)
    {
        current = grid.Count;
        dropSand(grid);
        next = grid.Count;
    }

    return grid.Values.Where(v => v is 'o').Count();
}

In [10]:
// Once all 24 units of sand shown above have come to rest, all further sand
// flows out the bottom

var testAnswer = DropAndCountSand(testInputLines, DropSand);
Console.WriteLine(testAnswer);

24


In [11]:
// How many units of sand come to rest before sand starts flowing into the abyss
// below?

var part1Answer = DropAndCountSand(inputLines, DropSand);
Console.WriteLine(part1Answer);

1406


In [12]:
// 1406 is correct!
Ensure(1406, part1Answer);

### --- Part Two ---

Puzzle description redacted as-per Advent of Code guidelines

You may find the puzzle description at: https://adventofcode.com/2022/day/14

Ok, I think we're pretty well set up for this one. We just need to tweak our falling simulation slightly so that it always stops on the new virtual floor.Ultimately, the initial block of sand will have no further options and will just overwrite the existing sand tile at the origin, adding no new items and terminating the simulation as-before.

In [14]:
void DropSand2(SandGrid grid)
{
    Point sand = (500, 0);

    var floor = floorCache.TryGetValue(grid, out var f) switch {
        true => f,
        _ => grid.Where(gg => gg.Value is Rock).Max(gg => gg.Key.Y) + 2
    };
    floorCache[grid] = floor;

    while (!grid.ContainsKey(sand))
    {
        // See if we can drop down
        var next = directions.Select(dir => sand + dir)
            .FirstOrDefault(n => !grid.ContainsKey(n) && n.Y < floor);
        
        // Nowhere left to fall?
        if (next == default) { break; }

        // Still falling
        sand = next;
    }

    grid[sand] = Sand;
}

Dictionary<SandGrid, int> floorCache = new();

In [15]:
// In the example above, the situation finally looks like this after 93 units of
// sand come to rest:

var part2TestAnswer = DropAndCountSand(testInputLines, DropSand2);
Console.WriteLine(part2TestAnswer);

93


In [16]:
// Using your scan, simulate the falling sand until the source of the sand
// becomes blocked. How many units of sand come to rest?

var part2Answer = DropAndCountSand(inputLines, DropSand2);
Console.WriteLine(part2Answer);

20870


In [17]:
// 20870 is correct!
Ensure(20870, part2Answer);